In [1]:
#First, the point cloud has to be loaded and then read.
#Its probably way too big to work with from the start, so I think I will use voxel downsampling
#But first, imports

import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import os   
from pyntcloud import PyntCloud

#Loading the point cloud

initialCloud =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "croppedCloud.ply"))




Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [3]:
#Making a voxelgrid from the point cloud
voxel_size = 3
voxel_grid = o3d.geometry.VoxelGrid.create_from_point_cloud(initialCloud, voxel_size)



In [14]:
#Extracting center points so we can calculate PCA for the points within each voxel

voxel_center_coordinates= []

for i in voxel_grid.get_voxels():
    voxel_center_coordinates.append(voxel_grid.get_voxel_center_coordinate(i.grid_index))



In [38]:

from sklearn.decomposition import PCA
from tqdm.auto import tqdm

#Now we have to extract the points from the point cloud that are within each voxel
voxel_size = 3


linearity =[]
planarity = []
scattering = []
omnivariance = []
sum_eigenvalues = []
anisotropy = []
eigentropy = []
change_of_curvature = []
points_per_voxel = []

#Run this to just run it on the first 100 voxels
#voxel_center_coordinates = voxel_center_coordinates[:100]

for center in tqdm(voxel_center_coordinates):
    minboundx = center[0] - voxel_size/2
    minboundy = center[1] - voxel_size/2
    minboundz = center[2] - voxel_size/2

    maxboundx = center[0] + voxel_size/2
    maxboundy = center[1] + voxel_size/2
    maxboundz = center[2] + voxel_size/2

    voxel_extracted_points =initialCloud.crop(o3d.geometry.AxisAlignedBoundingBox(min_bound=(minboundx, minboundy, minboundz), max_bound=(maxboundx, maxboundy, maxboundz)))
    #print("Number of points in voxel ", len(voxel_extracted_points.points))
    
    points = np.asarray(voxel_extracted_points.points)

    #Checking if there are enough points in the voxel to calculate PCA
    
    if len(points) < 15:
        linearity.append(0)
        planarity.append(0)
        scattering.append(0)
        omnivariance.append(0)
        sum_eigenvalues.append(0)
        anisotropy.append(0)
        eigentropy.append(0)
        change_of_curvature.append(0)
        points_per_voxel.append(points) #Kan her appende 0 istedenfor points?
        continue
    
    pca = PCA(n_components=3)
    pca.fit(points)

    # Extract the eigenvalues and eigenvectors
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    
    #Calculating the features
    l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0] 
    p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
    s = eigenvalues[2] / eigenvalues[0]
    o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
    s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
    a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
    e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
    c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])
    
    
    linearity.append(l)
    planarity.append(p)
    scattering.append(s)
    omnivariance.append(o)
    sum_eigenvalues.append(s_e)
    anisotropy.append(a)
    eigentropy.append(e)
    change_of_curvature.append(c_o_c)
    points_per_voxel.append(points)
   
    

  0%|          | 0/29528 [00:00<?, ?it/s]

In [39]:
print(len(linearity))
print(len(points_per_voxel))

29528
29528


In [40]:
#Making a function to extract points based on features and thresholds to test things out

def extract_points_based_on_feature(feature, feature_lower_treshold, feature_upper_treshold, points_per_voxel):
    points = []
    for i in range(len(feature)):
        if feature_lower_treshold < feature[i] < feature_upper_treshold:
            points.append(points_per_voxel[i])
    return points



In [52]:
#Function to test out the extraction of points based on features

extracted_points = extract_points_based_on_feature(eigentropy, 0.75, 1, points_per_voxel)

extracted_cloud = o3d.geometry.PointCloud()

for i in tqdm(extracted_points):
    extracted_cloud.points.extend(i)
    
o3d.visualization.draw_geometries([extracted_cloud])


  0%|          | 0/4123 [00:00<?, ?it/s]

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Ugyldig referanse. 
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: Forespurt transformeringsoperasjon er ikke støttet. 


In [88]:
#Creating a function to calculate the features for a given set of points


def calculateFeatures(points):
    pts = np.asarray(points) 
    pca = PCA(n_components=3)
    pca.fit(pts)
    
    # Extract the eigenvalues and eigenvectors
    eigenvalues = pca.explained_variance_
    eigenvectors = pca.components_
    
    #Calculating the features
    l = (eigenvalues[0] - eigenvalues[1]) / eigenvalues[0]
    p = (eigenvalues[1] - eigenvalues[2]) / eigenvalues[0]
    s = eigenvalues[2] / eigenvalues[0]
    o = (eigenvalues[0]*eigenvalues[1]*eigenvalues[2])**(1/3)
    s_e = eigenvalues[0] + eigenvalues[1] + eigenvalues[2]
    a = (eigenvalues[0] - eigenvalues[2]) / eigenvalues[0]
    e = (-1)*(eigenvalues[0]*np.log(eigenvalues[0]) + eigenvalues[1]*np.log(eigenvalues[1]) + eigenvalues[2]*np.log(eigenvalues[2]))
    c_o_c = eigenvalues[2]/(eigenvalues[0]+eigenvalues[1]+eigenvalues[2])
    
    return l, p, s, o, s_e, a, e, c_o_c, eigenvalues, eigenvectors



In [91]:
#Calculating features for 3 houses
house =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedHouse.ply"))
house2 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedHouse2.ply"))
house3 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedHouse3.ply"))


houseFeatures = calculateFeatures(house.points)
houseFeatures2 = calculateFeatures(house2.points)
houseFeatures3 = calculateFeatures(house3.points)


print("Linearity :" ,houseFeatures[0], houseFeatures2[0], houseFeatures3[0])
print("Planarity :" ,houseFeatures[1], houseFeatures2[1], houseFeatures3[1])
print("Scattering :" ,houseFeatures[2], houseFeatures2[2], houseFeatures3[2])
print("Omnivariance :" ,houseFeatures[3], houseFeatures2[3], houseFeatures3[3])
print("Sum of eigenvalues :" ,houseFeatures[4], houseFeatures2[4], houseFeatures3[4])
print("Anisotropy :" ,houseFeatures[5], houseFeatures2[5], houseFeatures3[5])
print("Eigenentropy :" ,houseFeatures[6], houseFeatures2[6], houseFeatures3[6])
print("Change of curvature :" ,houseFeatures[7], houseFeatures2[7], houseFeatures3[7])




#Making a scatterplot for each feature




Linearity : 0.17789106441625202 0.5544561822376141 0.56829661179632
Planarity : 0.4369548386017082 0.33552532841917787 0.3752820317081494
Scattering : 0.3851540969820398 0.1100184893432081 0.05642135649553062
Omnivariance : 31.216447269184073 35.6311781014578 56.593705463876205
Sum of eigenvalues : 101.09187869547794 151.44870359283362 290.5345652100888
Anisotropy : 0.6148459030179603 0.8899815106567919 0.9435786435044694
Eigenentropy : -362.39741418251685 -634.6838816007335 -1429.8709478585688
Change of curvature : 0.17449397344109246 0.07072586475042421 0.03791439978167614


In [92]:
#Calculating features for 3 trees
tree =  o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedTree.ply"))
tree2 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedTree2.ply"))
tree3 = o3d.io.read_point_cloud(os.path.join(os.getcwd(), "data", "extractedTree3.ply"))


treeFeatures = calculateFeatures(tree.points)
treeFeatures2 = calculateFeatures(tree2.points)
treeFeatures3 = calculateFeatures(tree3.points)


print("Linearity :" ,treeFeatures[0], treeFeatures2[0], treeFeatures3[0])
print("Planarity :" ,treeFeatures[1], treeFeatures2[1], treeFeatures3[1])
print("Scattering :" ,treeFeatures[2], treeFeatures2[2], treeFeatures3[2])
print("Omnivariance :" ,treeFeatures[3], treeFeatures2[3], treeFeatures3[3])
print("Sum of eigenvalues :" ,treeFeatures[4], treeFeatures2[4], treeFeatures3[4])
print("Anisotropy :" ,treeFeatures[5], treeFeatures2[5], treeFeatures3[5])
print("Eigenentropy :" ,treeFeatures[6], treeFeatures2[6], treeFeatures3[6])
print("Change of curvature :" ,treeFeatures[7], treeFeatures2[7], treeFeatures3[7])



Linearity : 0.42906009838900616 0.2229921316247172 0.260953560855383
Planarity : 0.10786984629037658 0.12603996336759488 0.19781701911389216
Scattering : 0.4630700553206173 0.650967905007688 0.5412294200307248
Omnivariance : 15.137927881908917 24.738396968984496 34.626539094443835
Sum of eigenvalues : 47.9741337765976 75.38545701249012 107.16314091022525
Anisotropy : 0.5369299446793827 0.3490320949923121 0.45877057996927517
Eigenentropy : -135.67360274987504 -244.22734412966065 -386.466190837289
Change of curvature : 0.2276636128267424 0.26811136756141307 0.23735260707731196
